### Find the maximum impacts for individual, local, synergistic and global nudges

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json

import numpy as np
from scipy import stats

import probability_distributions
import maximum_nudges
import evolutionary_algorithms as ea
import maximum_nudges_evolutionary as ev_max_nudges

### Notebook level constants

In [17]:
NUDGE_SIZE = 0.01
NUMBER_OF_STATES = 5

#### First generate a generic input distribution

In [18]:
#distribution parameters
input_variables = 2
number_of_states = 5

#generate both input and conditional output with Dirichlet weights
distribution_shape = [number_of_states]*input_variables
total_number_of_states = reduce(lambda x,y: x*y, distribution_shape)
input_dist = np.random.dirichlet([1]*total_number_of_states)
input_dist = np.reshape(input_dist, distribution_shape)
cond_shape = [number_of_states]*(input_variables+1)
cond_output = [
    probability_distributions.compute_joint_uniform_random((number_of_states,))
    for i in range(number_of_states**(input_variables))
]
cond_output = np.array(cond_output)
cond_output = np.reshape(cond_output, cond_shape)


#### load the generated input and conditional output distribution

In [19]:
PATH = "/home/derkjan/Documents/academics_UVA/master_thesis/code/"
INPUT_FOLDER = "input_distributions/"
COND_OUTPUT_FOLDER = "conditional_output_distributions/"
FOLDER_FORMAT_INPUT = "{}var_{}states/"
FOLDER_FORMAT_CONDITIONAL = "{}var_{}states/"
FILE_FORMAT_INPUT = "dist_{}.npy"
FILE_FORMAT_COND_OUTPUT = "cond_dist_{}.npy"

DIRICHLET_FOLDER_INPUT = "dirichlet/"
ENTROPY_LOW_FOLDER_INPUT = "entropy_0.5/"
ENTROPY_MEDIUM_FOLDER_INPUT = "entropy_0.75/"

DIRICHLET_FOLDER_COND_OUTPUT = "dirichlet/"


def generate_distributions(path_to_files, file_format, number_of_distributions):
    for i in range(number_of_distributions):
        file_name = path_to_files + file_format.format(i)
        with open(file_name, 'rb') as f:
            yield np.load(f)
            
def generate_input_and_conditional_output(input_type, parameters, cond_output_type="dirichlet"):
    """
    
    Parameters:
    ----------
    input_type: string in set {"dirichlet", "entropy_0.5", "entropy_0.75"}
    parameters: dict
    cond_output: 
    
    Returns: a dict with keys:
    -------
    number_of_var: a number
    number_of_states: a number
    input_dist: nd-array
    cond_output: nd-array
    
    """
    if input_type == "dirichlet":
        input_dirichlet_path = PATH + INPUT_FOLDER + DIRICHLET_FOLDER_INPUT
    elif input_type == "entropy_0.75":
        input_dirichlet_path = PATH + INPUT_FOLDER + ENTROPY_MEDIUM_FOLDER_INPUT
    elif input_type == "entropy_0.5":
        input_dirichlet_path = PATH + INPUT_FOLDER + ENTROPY_LOW_FOLDER_INPUT
    else:
        raise ValueError("supply valid input distribution type")

    cond_output_dirichlet_path = PATH + COND_OUTPUT_FOLDER + DIRICHLET_FOLDER_COND_OUTPUT

    min_inputs = parameters["min_number_inputs"]
    max_inputs = parameters["max_number_inputs"]
    number_of_states = parameters["number_of_states"]
    for number_of_var in range(min_inputs, max_inputs, 1):
        path_to_input_files = (
            input_dirichlet_path 
            + FOLDER_FORMAT_INPUT.format(number_of_var, number_of_states)
        )
        path_to_cond_output_files = (
            cond_output_dirichlet_path 
            + FOLDER_FORMAT_CONDITIONAL.format(number_of_var, number_of_states)
        )
        input_generator = generate_distributions(
            path_to_input_files, FILE_FORMAT_INPUT, 
            parameters["number_of_distributions"]
        )
        cond_output_generator = generate_distributions(
            path_to_cond_output_files, FILE_FORMAT_COND_OUTPUT, 
            parameters["number_of_distributions"]
        )
        input_shape = [number_of_states]*number_of_var
        cond_output_shape = [number_of_states]*(number_of_var+1)
        for sample in range(parameters["number_of_distributions"]):
            input_dist = next(input_generator)
            input_dist = np.reshape(input_dist, input_shape)
            cond_output = next(cond_output_generator)
            cond_output = np.reshape(cond_output, cond_output_shape)
            yield {
                "number_of_vars": number_of_var,
                "number_of_states": parameters["number_of_states"],
                "input_dist": input_dist,
                "cond_output": cond_output
            }
            

In [20]:
parameters_distributions = {
    "max_number_inputs": 6,
    "min_number_inputs": 1,
    "number_of_states": 5,
    "number_of_distributions": 100
}

generator = generate_input_and_conditional_output(
    'dirichlet', parameters_distributions, cond_output_type="dirichlet"
)

for dist_dict in generator:
    a = dist_dict

#### define the function to find max impact local nudges

In [21]:
def find_optimum_local_nudge(input_dist, cond_output, number_of_input_variables, 
                             number_of_states, nudge_size, parameters):
    """optimize local nudge
    
    Parameters:
    ----------
    input_dist:nd-array
    cond_output: nd-array, one axis more than input_dist
    
    """
    local_nudges = []
    for _ in range(parameters["population_size"]):
        new_local_nudge = ev_max_nudges.LocalNudge.create_local_nudge(
            parameters["nudged_vars_to_states"], nudge_size, 
            parameters["mutation_size_weights"], parameters["start_mutation_size"],
            parameters["change_mutation_size"], timestamp=0
        )
        local_nudges.append(new_local_nudge)

    for local_nudge in local_nudges:
        local_nudge.evaluate(input_dist, cond_output)

    initial_impact = ea.sort_individuals(local_nudges)[0].score

    #start the optimization process
    find_max_local_nudge = ev_max_nudges.FindMaximumLocalNudge(
        input_dist, cond_output, nudge_size, 
        parameters["generational"], parameters["number_of_children"], 
        parameters["parent_selection_mode"]
    )
    max_local_nudge_individual = find_max_local_nudge.get_max_nudge(
        local_nudges, parameters["number_of_generations"]
    )
    max_impact = max_local_nudge_individual.score 
    print("local nudge: initial impact {} max impact {}".format(initial_impact, max_impact))
    return max_impact


In [27]:
ev_max_nudges.TEST = False

parameters = {
    "number_of_generations": 250, 
    "population_size": 10,
    "number_of_children": 20, 
    "generational": True,
    "mutation_size": NUDGE_SIZE/4,
    "parent_selection_mode": "rank_exponential",
    "mutation_size_weights": 0.025,
    "start_mutation_size": NUDGE_SIZE/7.5,
}
parameters["change_mutation_size"] = parameters["start_mutation_size"]/12
parameters["nudged_vars_to_states"] = None

parameters_distributions = {
    "max_number_inputs": 6,
    "min_number_inputs": 2,
    "number_of_states": 5,
    "number_of_distributions": 100
}

generator = generate_input_and_conditional_output(
    'dirichlet', parameters_distributions, cond_output_type="dirichlet"
)

#change file name before turning run on
RUN = False
FILE_NAME = "max_impact_local_nudges_dirichlet2.json"
if RUN:
    max_local_impact_dict = {}
    prev_number_of_vars = -1
    for count, dist_dict in enumerate(generator):
        number_of_vars = dist_dict["number_of_vars"]
        number_of_states = dist_dict["number_of_states"]
        if number_of_vars != len(dist_dict["input_dist"].shape):
            print("WARNING in sample {} input dist has weird distribution".format(count))
        if number_of_vars != prev_number_of_vars:
            prev_number_of_vars = number_of_vars 
            print("the number of vars {}".format(number_of_vars))
        #print(dist_dict["cond_output"].shape)
        parameters["nudged_vars_to_states"] = {
            nudged_var:number_of_states for nudged_var in range(number_of_vars)
        }
        max_impact = find_optimum_local_nudge(
            dist_dict["input_dist"], dist_dict["cond_output"], number_of_vars,
            number_of_states, NUDGE_SIZE, parameters=parameters
        )
        if number_of_vars in max_local_impact_dict:
            max_local_impact_dict[number_of_vars].append(max_impact)
        else:
            max_local_impact_dict[number_of_vars] = [max_impact]

        if (count+1)%5==0 and count != 0:
            with open(FILE_NAME, 'w') as f:
                json.dump(max_local_impact_dict, f, indent=4)
                
    with open(FILE_NAME, 'w') as f:
        json.dump(max_local_impact_dict, f, indent=4)
            
print(max_local_impact_dict)



{2: [-0.0029856260217090147, -0.0036625968978055978]}


#### Find the maximum individual impact evolutionary

#### Find the maximum impact of an individual nudge exactly

Due to different definitions of nudge size used, the nudge size needs to be divided by 2.

In [28]:
def find_max_impact_individual_nudge_exactly(input_dist, cond_output, nudge_size):
    new_input_dist = np.copy(input_dist)
    max_impacts = []
    for i in range(len(new_input_dist.shape)):
        new_input_dist = np.swapaxes(new_input_dist, i,
                                     len(new_input_dist.shape)-1)
        max_impact = maximum_nudges.find_maximum_local_nudge(
            new_input_dist, cond_output, nudge_size/2
        )
        max_impacts.append(max_impact)
        new_input_dist = np.swapaxes(new_input_dist, i,
                                     len(new_input_dist.shape)-1)
        
        return max(max_impacts)
    
max_impact = find_max_impact_individual_nudge_exactly(input_dist, cond_output, NUDGE_SIZE/2)
print("the actual maximum individual nudge {}".format(max_impact))

the actual maximum individual nudge 0.00170966015807


#### Run the experiment

In [30]:
parameters_distributions = {
    "max_number_inputs": 7,
    "min_number_inputs": 1,
    "number_of_states": 5,
    "number_of_distributions": 100
}

generator = generate_input_and_conditional_output(
    'dirichlet', parameters_distributions, cond_output_type="dirichlet"
)

#set new file name before turning run on
RUN = False
FILE_NAME = "max_impact_individual_nudges_dirichlet2_first100samples.json"
if RUN:
    max_individual_impact_dict = {}
    prev_number_of_vars = -1
    for count, dist_dict in enumerate(generator):
        print(count)
        number_of_vars = dist_dict["number_of_vars"]
        number_of_states = dist_dict["number_of_states"]
        if number_of_vars != len(dist_dict["input_dist"].shape):
            print("WARNING in sample {} input dist has weird distribution".format(count))
        if number_of_vars != prev_number_of_vars:
            prev_number_of_vars = number_of_vars 
            print("the number of vars {}".format(number_of_vars))
        #print(dist_dict["cond_output"].shape)
        max_impact = find_max_impact_individual_nudge_exactly(
            dist_dict["input_dist"], dist_dict["cond_output"], NUDGE_SIZE/2
        )
        if number_of_vars in max_individual_impact_dict:
            max_individual_impact_dict[number_of_vars].append(max_impact)
        else:
            max_individual_impact_dict[number_of_vars] = [max_impact]

        if (count+1)%5==0 and count != 0:
            with open(FILE_NAME, 'w') as f:
                json.dump(max_individual_impact_dict, f, indent=4)

    with open(FILE_NAME, 'w') as f:
        json.dump(max_individual_impact_dict, f, indent=4)

    print(max_individual_impact_dict)


0
the number of vars 1
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
the number of vars 2
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
the number of vars 3
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261

### Find the maximum global nudges

In [13]:
_, _, max_global_nudge_impact = maximum_nudges.find_max_control_impact(input_dist, cond_output, NUDGE_SIZE/2)
print(max_global_nudge)

In [ ]:
parameters_distributions = {
    "max_number_inputs": 7,
    "min_number_inputs": 2,
    "number_of_states": 5,
    "number_of_distributions": 200
}

generator = generate_input_and_conditional_output(
    'dirichlet', parameters_distributions, cond_output_type="dirichlet"
)

#change file name before turning RUN on
RUN = False
FILE_NAME = "max_impact_global_nudges_dirichlet_first200samples2.json"
if RUN:
    max_global_impact_dict = {}
    prev_number_of_vars = -1
    for count, dist_dict in enumerate(generator):
        print(count)
        number_of_vars = dist_dict["number_of_vars"]
        number_of_states = dist_dict["number_of_states"]
        if number_of_vars != len(dist_dict["input_dist"].shape):
            print("WARNING in sample {} input dist has weird distribution".format(count))
        if number_of_vars != prev_number_of_vars:
            prev_number_of_vars = number_of_vars 
            print("the number of vars {}".format(number_of_vars))
        #print(dist_dict["cond_output"].shape)
        _, _, max_global_nudge_impact = maximum_nudges.find_max_control_impact(
            dist_dict["input_dist"], dist_dict["cond_output"], NUDGE_SIZE/2
        )

        if number_of_vars in max_global_impact_dict:
            max_global_impact_dict[number_of_vars].append(max_global_nudge_impact)
        else:
            max_global_impact_dict[number_of_vars] = [max_global_nudge_impact]

        if (count+1)%5==0 and count != 0:
            with open(FILE_NAME, 'w') as f:
                json.dump(max_global_impact_dict, f, indent=4)

    with open(FILE_NAME, 'w') as f:
        json.dump(max_global_impact_dict, f, indent=4)

    print(max_global_impact_dict)
